## VGG16 :

is a convolutional neural network architecture that was introduced by the Visual Geometry Group (VGG) at the University of Oxford. It is known for its simplicity and effectiveness in image classification tasks. Here's an overview of VGG16:

## Architecture: 
VGG16 consists of 16 convolutional layers, hence the name "VGG16". The architecture is characterized by its repeated use of 3x3 convolutional filters, followed by max-pooling layers. There are also a few fully connected layers at the end of the network.

## Convolutional Layers:
The convolutional layers in VGG16 consist of stacks of 3x3 convolutional filters with a stride of 1, followed by rectified linear activation (ReLU) functions. The number of filters increases as the spatial resolution decreases, leading to a gradual reduction in spatial dimensions and an increase in the number of feature maps.

## Max-Pooling Layers: 
After every two convolutional layers, VGG16 uses max-pooling layers with a 2x2 window and a stride of 2. Max-pooling reduces the spatial dimensions of the feature maps while retaining the most salient features.

## Fully Connected Layers:
The final layers of VGG16 consist of fully connected layers followed by softmax activation for classification. These layers take the flattened output of the last convolutional layer and transform it into predictions for each class in the classification task.

## Pre-Trained Models: 
VGG16 is often used as a pre-trained model for transfer learning. Pre-trained versions of VGG16, trained on large-scale image datasets like ImageNet, are available in frameworks like TensorFlow and Keras. These pre-trained models have learned rich feature representations from the original dataset and can be fine-tuned or used as feature extractors for other tasks.

## Applications: 
VGG16 has been widely used in various computer vision tasks, including image classification, object detection, and image segmentation. Its relatively simple architecture and strong performance make it a popular choice for baseline experiments and as a backbone in more complex models.

While VGG16 achieved impressive results when it was introduced, newer architectures like ResNet, Inception, and EfficientNet have since surpassed it in terms of accuracy and computational efficiency. However, VGG16 remains a valuable tool for understanding and experimenting with deep learning in computer vision.

In [ ]:
# Import the required libarary
from tensorflow.keras import datasets , layers , models
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.models import save_model,Sequential
import numpy as np
import random
import pickle

In [ ]:
# Get the current directory
current_dir = os.getcwd()

# Get the parent directory (one level up)
current_dir = os.path.dirname(current_dir)

# Get the parent directory (one level up)
parent_dir = os.path.dirname(current_dir)

# Print the parent directory
print("Parent Directory:", parent_dir)

In [ ]:
dataset_dir = parent_dir+"/datasets/raw_dataset/Digital images of defective and good condition tyres" # dataset directory

## DATA PREPROCESSING and FEATURE ENGINEERING

In [ ]:
# Define parameters for preprocessing
batch_size = 32
image_size = (160, 160)

In [ ]:
#Load the data with the help of the tensorflow keras utils model :
dataset =  tf.keras.utils.image_dataset_from_directory(dataset_dir ,image_size=image_size)                                                                                 
class_name = dataset.class_names

In [ ]:
# Data augumenation
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip('horizontal_and_vertical'),
                                         tf.keras.layers.RandomRotation(0.2),
                                         tf.keras.layers.RandomZoom(0.2)])


In [ ]:
# Initiate the object of mobilenet inbuild processess input module
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [ ]:
# Split the dataset in to train and validation set

In [ ]:
val_batches = tf.data.experimental.cardinality(dataset)
print("%d"%val_batches)
validation_dataset = dataset.take(val_batches//4)
train_dataset = dataset.skip(val_batches//4)

In [ ]:
# SPlit the dataset into test and validation dataset

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
print("%d"%val_batches)

test_dataset = validation_dataset.take(val_batches//2)
validation_dataset = validation_dataset.skip(val_batches//2)

In [ ]:
## Autotune : In TensorFlow, tf.data.AUTOTUNE is a special constant that can be used when configuring input pipelines for better 
## performance. It's particularly useful when dealing with input data pipelines that involve I/O operations. 

In [ ]:
AUTOTUNE =  tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset =  validation_dataset.prefetch(buffer_size=AUTOTUNE)

## Feature Engineering

In [ ]:
# Define the base model as mobilenetv2

IMG_SHAPE = (160,160)+(3,)
base_model = tf.keras.applications.VGG16(input_shape = IMG_SHAPE,include_top=False,weights='imagenet')

In [ ]:
# process all the train dataset to basemodel and prepare the batch
image_batch , label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# Setting base_model.trainable = False means that freezing the weights of the pre-trained model (base_model) during training. 
#In other words, the model's weights will not be updated during the training process when this flag is set to False.

In [ ]:
base_model.trainable = False

In [ ]:
# Model Summary
base_model.summary()

In [ ]:
# define average pooling layer for feature pooling and dimensionality reduction
global_average_layer =  tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
# Prediction layer with 1 neuron as output of binary classification

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch =prediction_layer(feature_batch_average)
print(prediction_batch.shape)

## Model Building

In [ ]:
inputs = tf.keras.layers.Input(shape=(image_size+(3,)))
x = data_augmentation(inputs)
x = preprocess_input(x)         
x = base_model(x,training = False)
x=  global_average_layer(x)
x=  tf.keras.layers.Dropout(0.2)(x)
output=prediction_layer(x)
models = tf.keras.Model(inputs,output)

In [ ]:
models.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
              ,metrics=['accuracy'])

In [ ]:
# Model Summary

In [ ]:
models.summary()

In [ ]:
# Validate the accuracy before training
loss0,accuracy0= models.evaluate(validation_dataset)

In [ ]:
# Time to train the model

## Model Training

In [ ]:
intial_epochs = 5
history = models.fit(train_dataset,epochs=intial_epochs,validation_data=validation_dataset)

In [ ]:
# Plot the accuracy of trained model
plt.plot(history.history['accuracy'],label= 'accuracy')
plt.plot(history.history['val_accuracy'],label= 'val_accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.title("VGG16_accuracy_plot")
plt.savefig(parent_dir+'\\visuals\\vgg16_accuracy_plot.png')

In [ ]:
# Plot the loss of trained model
plt.plot(history.history['loss'],label= 'loss')
plt.plot(history.history['val_loss'],label= 'val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.title("vgg16_loss_plot")
plt.savefig(parent_dir+'\\visuals\\vgg16_loss_plot.png')

## Fine Tunning

In [ ]:
base_model.trainable = True

In [ ]:
print("number of layers in base models" , len(base_model.layers))

In [ ]:
# Model compilation with RMSprop optimizer this time

In [ ]:
models.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001/10),loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
              ,metrics=['accuracy'])

In [ ]:
models.summary()

In [ ]:
# Start model traing on base model 

In [ ]:
fine_tune_epoch = 5
totol_epoch =  fine_tune_epoch+intial_epochs

history = models.fit(train_dataset,epochs=totol_epoch,initial_epoch=history.epoch[-1],validation_data=validation_dataset)

In [ ]:
# Plot the accuracy of trained model
plt.plot(history.history['accuracy'],label= 'accuracy')
plt.plot(history.history['val_accuracy'],label= 'val_accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.title("vgg16_accuracy_plot")
plt.savefig(parent_dir+'\\visuals\\vgg16_finetune_accuracy_plot.png')

In [ ]:
# Plot the loss of trained model
plt.plot(history.history['loss'],label= 'loss')
plt.plot(history.history['val_loss'],label= 'val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.title("vgg16_loss_plot")
plt.savefig(parent_dir+'\\visuals\\vgg16_finetune_loss_plot.png')

In [ ]:
# Save the data in form of tensorflow object
tf.data.Dataset.save(test_dataset, parent_dir+'/datasets/processed_dataset/vgg16_test_datasets.tfrecord')

In [ ]:
# Save the model
save_model(models , parent_dir+"/models/vgg16_model.keras")

In [ ]:
with open(parent_dir+'/models/vgg16_training_history.pkl', 'wb') as file:
    pickle.dump(history, file)